# Triangle Configuration

This tutorial covers how to customize Triangle behavior for your project using `.triangle.toml`.

## Overview

Triangle uses a configuration file to adapt to each project's:
- **Verification strategy** - How to check and fix code
- **Code style** - Formatter, linter, type checker
- **Commit conventions** - Message format, issue linking

If no `.triangle.toml` exists, Triangle uses sensible defaults.

## Configuration File Structure

Create `.triangle.toml` in your project root:

```toml
# .triangle.toml

[verification]
fix_command = "./scripts/fix.sh"
check_command = "./scripts/check.sh"
fallback_tools = ["ruff", "black", "pyright"]

[style]
formatter = "black"
linter = "ruff"
type_checker = "pyright"
guidelines_file = "CONTRIBUTING.md"
line_length = 88

[commits]
convention = "conventional"
link_pattern = "Closes #{issue}"
```

## Loading Configuration

Triangle loads configuration from the **current working directory**, not the package location.

This means:
1. Each project has its own `.triangle.toml`
2. The config travels with your repo, not with the installed package
3. Different projects can have different verification strategies

In [1]:
from agent_workshop.agents.software_dev.config import (
    load_triangle_config,
    clear_config_cache,
    TriangleConfig,
)
from pathlib import Path

# Find the project root (where .triangle.toml lives)
# When running from notebooks/tutorials/, we need to go up 2 levels
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent if notebook_dir.name == "tutorials" else notebook_dir

print(f"Loading config from: {project_root}")
print(f".triangle.toml exists: {(project_root / '.triangle.toml').exists()}")
print()

# Load config from project root
config = load_triangle_config(project_root)

print("Verification Config:")
print(f"  fix_command: {config.verification.fix_command}")
print(f"  check_command: {config.verification.check_command}")
print(f"  fallback_tools: {config.verification.fallback_tools}")

print("\nStyle Config:")
print(f"  formatter: {config.style.formatter}")
print(f"  linter: {config.style.linter}")
print(f"  type_checker: {config.style.type_checker}")
print(f"  guidelines_file: {config.style.guidelines_file}")

print("\nCommit Config:")
print(f"  convention: {config.commits.convention}")
print(f"  link_pattern: {config.commits.link_pattern}")

Loading config from: /home/trentleslie/Insync/projects/agent-workshop
.triangle.toml exists: True

Verification Config:
  fix_command: ./scripts/fix.sh
  check_command: ./scripts/check.sh
  fallback_tools: ['ruff', 'black', 'pyright']

Style Config:
  formatter: black
  linter: ruff
  type_checker: pyright
  guidelines_file: dev_guidelines.md

Commit Config:
  convention: conventional
  link_pattern: Closes #{issue}


## Section 1: Verification

The `[verification]` section controls how Triangle checks and fixes generated code.

### Options

| Option | Type | Default | Description |
|--------|------|---------|-------------|
| `fix_command` | string | `null` | Script to auto-fix issues |
| `check_command` | string | `null` | Script to run all checks |
| `fallback_tools` | list | `["ruff", "black", "pyright"]` | Tools to run if scripts don't exist |

### Script-Based Verification

Create scripts in your project:

**`scripts/fix.sh`**:
```bash
#!/bin/bash
set -e

# Auto-fix formatting and linting
ruff format .
ruff check --fix .
```

**`scripts/check.sh`**:
```bash
#!/bin/bash
set -e

# Run all checks
ruff check .
pyright .
pytest tests/ -x
```

Then configure:
```toml
[verification]
fix_command = "./scripts/fix.sh"
check_command = "./scripts/check.sh"
```

### Fallback Behavior

If scripts don't exist or aren't specified, Triangle falls back to running tools directly:

```toml
[verification]
# No scripts - Triangle runs these tools directly
fallback_tools = ["ruff", "black", "pyright"]
```

Triangle will detect which tools are available and run them.

In [2]:
# Example: Check if verification scripts exist
import subprocess

def check_verification_setup(config, project_root):
    """Check what verification strategy will be used."""
    print("Verification Strategy Analysis:")
    print("=" * 50)
    
    if config.verification.check_command:
        # Resolve script path relative to project root
        script_path = project_root / config.verification.check_command.lstrip("./")
        if script_path.exists():
            print(f"✅ Check script exists: {script_path}")
        else:
            print(f"⚠️  Check script configured but missing: {script_path}")
            print(f"   Fallback: {config.verification.fallback_tools}")
    else:
        print("ℹ️  No check script configured")
        print(f"   Using fallback tools: {config.verification.fallback_tools}")
    
    print()
    
    # Check which fallback tools are available
    print("Available fallback tools:")
    for tool in config.verification.fallback_tools:
        result = subprocess.run(["which", tool], capture_output=True)
        status = "✅" if result.returncode == 0 else "❌"
        print(f"  {status} {tool}")

check_verification_setup(config, project_root)

Verification Strategy Analysis:
✅ Check script exists: /home/trentleslie/Insync/projects/agent-workshop/scripts/check.sh

Available fallback tools:
  ✅ ruff
  ❌ black
  ❌ pyright


## Section 2: Style

The `[style]` section configures code style preferences.

### Options

| Option | Type | Default | Description |
|--------|------|---------|-------------|
| `formatter` | string | `"black"` | Code formatter |
| `linter` | string | `"ruff"` | Linter tool |
| `type_checker` | string | `"pyright"` | Type checker |
| `guidelines_file` | string | `null` | Project guidelines to inject into prompts |
| `line_length` | int | `88` | Maximum line length |

### Guidelines File

The `guidelines_file` is **injected into code generation prompts**. This ensures generated code follows your project's conventions.

```toml
[style]
guidelines_file = "CONTRIBUTING.md"
```

Your `CONTRIBUTING.md` might include:
- Naming conventions
- Architecture patterns
- Testing requirements
- Documentation standards

Triangle reads this file and includes it in the prompt when generating code.

In [3]:
# Example: Show what guidelines would be injected
def show_guidelines(config, project_root):
    """Display guidelines that would be injected into prompts."""
    if not config.style.guidelines_file:
        print("No guidelines file configured.")
        return
    
    guidelines_path = project_root / config.style.guidelines_file
    if not guidelines_path.exists():
        print(f"Guidelines file configured: {config.style.guidelines_file}")
        print(f"Full path: {guidelines_path}")
        print(f"⚠️  File not found - create it to inject guidelines into prompts")
        return
    
    content = guidelines_path.read_text()
    print(f"Guidelines from {guidelines_path}:")
    print("=" * 50)
    # Show first 500 chars
    print(content[:500])
    if len(content) > 500:
        print(f"\n... ({len(content) - 500} more characters)")

show_guidelines(config, project_root)

Guidelines file configured: dev_guidelines.md
Full path: /home/trentleslie/Insync/projects/agent-workshop/dev_guidelines.md
⚠️  File not found - create it to inject guidelines into prompts


## Section 3: Commits

The `[commits]` section configures commit and PR conventions.

### Options

| Option | Type | Default | Description |
|--------|------|---------|-------------|
| `convention` | string | `"conventional"` | Commit message format |
| `link_pattern` | string | `"Closes #{issue}"` | Pattern for linking issues |

### Commit Conventions

Supported conventions:
- `"conventional"` - `type(scope): message` (e.g., `feat(auth): add login`)
- `"angular"` - Similar to conventional with Angular specifics
- `"none"` - No enforced format

### Issue Linking

The `link_pattern` is used in PR descriptions. `{issue}` is replaced with the issue number.

```toml
[commits]
link_pattern = "Closes #{issue}"
```

For issue #42, this produces: `Closes #42`

Other common patterns:
- `"Fixes #{issue}"` - Fixes keyword
- `"Resolves #{issue}"` - Resolves keyword
- `"Related to #{issue}"` - No auto-close

In [4]:
# Example: Generate a PR link
def generate_issue_link(config, issue_number: int) -> str:
    """Generate an issue link using the configured pattern."""
    return config.commits.link_pattern.format(issue=issue_number)

print(f"Convention: {config.commits.convention}")
print(f"Link pattern: {config.commits.link_pattern}")
print()
print("Example links:")
for issue in [1, 42, 100]:
    print(f"  Issue #{issue}: {generate_issue_link(config, issue)}")

Convention: conventional
Link pattern: Closes #{issue}

Example links:
  Issue #1: Closes #1
  Issue #42: Closes #42
  Issue #100: Closes #100


## Default Configuration

If no `.triangle.toml` exists, Triangle uses these defaults:

```python
TriangleConfig(
    verification=VerificationConfig(
        fix_command=None,
        check_command=None,
        fallback_tools=["ruff", "black", "pyright"],
    ),
    style=StyleConfig(
        formatter="black",
        linter="ruff",
        type_checker="pyright",
        guidelines_file=None,
        line_length=88,
    ),
    commits=CommitConfig(
        convention="conventional",
        link_pattern="Closes #{issue}",
    ),
)
```

This means you can use Triangle without any configuration for Python projects using common tools.

In [5]:
# Show default config (no .triangle.toml)
import tempfile

# Load from a directory without config
clear_config_cache()  # Clear cache first
with tempfile.TemporaryDirectory() as tmpdir:
    default_config = load_triangle_config(tmpdir)
    print("Default Configuration (no .triangle.toml):")
    print("=" * 50)
    print(default_config.model_dump_json(indent=2))

Default Configuration (no .triangle.toml):
{
  "verification": {
    "fix_command": null,
    "check_command": null,
    "fallback_tools": [
      "ruff",
      "black",
      "pyright"
    ]
  },
  "style": {
    "formatter": "black",
    "linter": "ruff",
    "type_checker": "pyright",
    "guidelines_file": null,
    "line_length": 88
  },
  "commits": {
    "convention": "conventional",
    "link_pattern": "Closes #{issue}"
  }
}


## Configuration Patterns

### Minimal Python Project

Just use defaults - no config needed!

### Python with Custom Scripts

```toml
[verification]
fix_command = "make fix"
check_command = "make check"
```

### Node.js Project

```toml
[verification]
fix_command = "npm run lint:fix"
check_command = "npm run lint && npm test"
fallback_tools = ["eslint", "prettier", "tsc"]

[style]
formatter = "prettier"
linter = "eslint"
type_checker = "tsc"
line_length = 100
```

### Monorepo with Guidelines

```toml
[verification]
check_command = "./scripts/ci.sh"

[style]
guidelines_file = "docs/CONTRIBUTING.md"

[commits]
convention = "conventional"
link_pattern = "Resolves #{issue}"
```

### Go Project

```toml
[verification]
fix_command = "go fmt ./..."
check_command = "go vet ./... && go test ./..."
fallback_tools = ["gofmt", "govet"]

[style]
formatter = "gofmt"
linter = "golint"
type_checker = "go"
```

## Programmatic Access

You can also create and use configurations programmatically:

In [6]:
from agent_workshop.agents.software_dev.config import (
    TriangleConfig,
    VerificationConfig,
    StyleConfig,
    CommitConfig,
)

# Create a custom config programmatically
custom_config = TriangleConfig(
    verification=VerificationConfig(
        check_command="pytest && ruff check",
        fallback_tools=["pytest", "ruff"],
    ),
    style=StyleConfig(
        formatter="ruff",
        linter="ruff",
        type_checker="mypy",
        line_length=100,
    ),
    commits=CommitConfig(
        convention="angular",
        link_pattern="Fixes #{issue}",
    ),
)

print("Custom Config:")
print(custom_config.model_dump_json(indent=2))

Custom Config:
{
  "verification": {
    "fix_command": null,
    "check_command": "pytest && ruff check",
    "fallback_tools": [
      "pytest",
      "ruff"
    ]
  },
  "style": {
    "formatter": "ruff",
    "linter": "ruff",
    "type_checker": "mypy",
    "guidelines_file": null,
    "line_length": 100
  },
  "commits": {
    "convention": "angular",
    "link_pattern": "Fixes #{issue}"
  }
}


## Caching Behavior

Triangle caches configuration to avoid re-reading the file on every operation.

If you modify `.triangle.toml` during a session, clear the cache:

In [7]:
from agent_workshop.agents.software_dev.config import clear_config_cache

# Modify .triangle.toml...

# Clear cache to reload
clear_config_cache()

# Now load_triangle_config() will re-read the file
print("Cache cleared. Next load will read from disk.")

Cache cleared. Next load will read from disk.


## Best Practices

1. **Commit `.triangle.toml`** to your repo - it's project configuration

2. **Keep scripts executable**:
   ```bash
   chmod +x scripts/check.sh scripts/fix.sh
   ```

3. **Use fallback_tools** as backup - ensures Triangle works even if scripts fail

4. **Add guidelines_file** for complex projects - improves code generation quality

5. **Match your CI** - use the same commands in `.triangle.toml` as your CI pipeline

6. **Start minimal** - you can always add more configuration later

## Next Steps

- **[08_triangle_quickstart.ipynb](./08_triangle_quickstart.ipynb)** - Basic Triangle usage
- **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - PR pipeline for code review
- **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Automated releases